In [12]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [13]:
db_benigh = np.array(pd.read_csv('benign_traffic.csv'))
db_attack = np.array(pd.read_csv('ack.csv'))

In [7]:
db_benigh = pd.read_csv('benign_traffic.csv')
db_attack = pd.read_csv('ack.csv')
trainset = db_benigh[1:round(len(db_benigh)*0.8)]
val_benigh = db_benigh[round(len(db_benigh)*0.8):round(len(db_benigh)*0.9)]
val_attack = db_attack[1:round(len(db_benigh)*0.1)]
test_benigh = db_benigh[round(len(db_benigh)*0.9):len(db_benigh)]
test_attack = db_attack[round(len(db_attack)*0.1):round(len(db_attack)*0.2)]
valset = pd.concat([val_benigh,val_attack])
testset = pd.concat([test_benigh,test_attack])
trainset = (trainset - trainset.mean()) / (trainset.std())
testset = (testset - testset.mean()) / (testset.std())
trainset = np.array(trainset)
valset = np.array(valset)
testset = np.array(testset)
testset = torch.Tensor(testset)
trainset = torch.Tensor(trainset)
testratio = 1 - abs((round(len(db_benigh)*0.9)-len(db_benigh))/len(testset))

In [9]:
print(len(test_benigh),
len(test_attack))

4955 10219


In [ ]:
batch_s = 32
trainset = torch.Tensor(db_benigh)
testset = torch.Tensor(db_attack[0:round(len(db_attack)*0.1)])
trainset = (trainset - trainset.mean()) / (trainset.std())
testset = (testset - testset.mean()) / (testset.std())
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_s,shuffle=False,num_workers=2)
testloader = torch.utils.data.DataLoader(testset,batch_size=1,shuffle=False,num_workers=2)

In [15]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()
        self.enc = nn.Sequential(
            nn.Linear(115, 64),
            nn.ReLU(),
            nn.Dropout(p = 0.1),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(p = 0.1),
            nn.Linear(32, 16),
            nn.ReLU()
        )
        self.dec = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Dropout(p = 0.1),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Dropout(p = 0.1),
            nn.Linear(64, 115),
            nn.ReLU()
        )
    def forward(self, x):
        encode = self.enc(x)
        decode = self.dec(encode)
        return decode

In [ ]:
autoencoder = AE()
autoencoder.train()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001)
loss_func = nn.MSELoss()
running_loss = 0.0
for epoch in range(50):
    
    #mini- batch loop
    epoch_loss = 0.0
    for idx, inp in enumerate(trainloader):
        optimizer.zero_grad() 
        decode = autoencoder(inp)
        loss = loss_func(decode, inp) 
        epoch_loss  += loss.item() / batch_s
        loss.backward() 
        optimizer.step()
    print( epoch , epoch_loss )
        
    running_loss += epoch_loss

0 38.25617795531866
1 18.957361772632794
2 15.549588491636769
3 21.892371786763988
4 26.1188965393153
5 6.150738970528664
6 13.005393012027298
7 10.660070579350304
8 9.921254428608336
9 13.289690179857644


In [10]:
threshold = 0
a = 0;
for idx, inp in enumerate(trainloader):
    i = max(sum(abs(autoencoder(inp) - inp)))
    print("inp is: ", inp)
    print("output is: ",autoencoder(inp))
    print("number is: ",idx)
    if i > threshold:
        threshold = i
    a += 1
    if idx > 3:
        break
print('threshold is', threshold, a)

NameError: name 'trainloader' is not defined

In [27]:
autoencoder.eval()
anmoaly = []
for idx, inp in enumerate(testloader):
    decode = autoencoder(inp)
    diff = torch.sum(abs(inp-decode))
    if diff > 0.3516:
        anmoaly.append(idx)
an_ratio = len(anmoaly)/len(testset)
print('The accuracy is ',an_ratio/testratio)

The accuracy is  1.0000978569331636


In [30]:
len(trainloader.dataset)

49548

In [31]:
torch.save(autoencoder.state_dict(), "vae_v1.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
testra